# Analyze guides

Since the guides can be given to us in a variety of orders and we need to order different ranges of sequence, this becomes rather tricky & I am having trouble working directly with R.

# imports & globals 🌎

In [50]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import regex as re
from pickle import dump,load
import sys

# Grab data & example

In [51]:
gene_data = pd.read_csv("../data/gene_data.csv")
print(gene_data.shape)

(47, 13)


In [52]:
guide_data = pd.read_excel("../inputs/47 genes and its guide sequence- synthego.xlsx")
print(guide_data.shape)

(162, 2)


In [53]:
guide_data[guide_data["Gene Name"] == "LYPLAL1"]

,Gene Name,Guides
129,LYPLAL1,A*U*A*CUGGGGUAACCGAGGGG
130,LYPLAL1,G*C*C*AUGCAGGAAGAUCAGAG
131,LYPLAL1,A*U*C*AGCGAUGGCGGCUGCGU


In [54]:
gene_data[gene_data.gene == "LYPLAL1"]

,Unnamed: 0,gene,organism,gene_link,gene_bank_url,ncbi_id,start,stop,strand,ncbi_phid,genbank_jquery,CDS,seq
40,40,LYPLAL1,Homo sapiens,https://www.ncbi.nlm.nih.gov/gene/127018,https://www.ncbi.nlm.nih.gov//nuccore/NC_00000...,568815597,219173878,219445496,off,CE8C9E9A32CEF9910000000001B900A1.m_23,https://www.ncbi.nlm.nih.gov/sviewer/viewer.fc...,"14..104,5270..5369,5523..5549,19205..19374,366...",agtggcatcagcgatggcggctgcgtcggggtcggttctgcagcgc...


In [55]:
guides = set(guide_data[guide_data["Gene Name"] == "LYPLAL1"].Guides)
guides

{'A*U*A*CUGGGGUAACCGAGGGG',
 'A*U*C*AGCGAUGGCGGCUGCGU',
 'G*C*C*AUGCAGGAAGAUCAGAG'}

In [56]:
seq = gene_data[gene_data.gene == "LYPLAL1"].seq.iloc[0]
cds = gene_data[gene_data.gene == "LYPLAL1"].CDS.iloc[0]
print(cds)
#print(seq)

14..104,5270..5369,5523..5549,19205..19374,36655..36770,37615..37851


# Perform the matching & the cuts

In [57]:
preprocess = lambda s: "".join(s.lower().split("*"))
def rna_2_dna(s):
    s = preprocess(s)
    s = s.replace("u","t")
    return s
[rna_2_dna(g) for g in guides]

['atactggggtaaccgagggg', 'gccatgcaggaagatcagag', 'atcagcgatggcggctgcgt']

In [58]:
def dna_to_complement(s):
    s = preprocess(s)
    s = s.replace("a","+")
    s = s.replace("t","a")
    s = s.replace("+","t")
    s = s.replace("g","+")
    s = s.replace("c","g")
    s = s.replace("+","c")
    return s
[dna_to_complement(rna_2_dna(g)) for g in guides]

['tatgaccccattggctcccc', 'cggtacgtccttctagtctc', 'tagtcgctaccgccgacgca']

In [59]:
def dna_to_rev(s):
    s = preprocess(s)
    return s[::-1]
[dna_to_rev(rna_2_dna(g)) for g in guides]

['ggggagccaatggggtcata', 'gagactagaaggacgtaccg', 'tgcgtcggcggtagcgacta']

In [60]:
def find_guide_loc(seq,guide,primer="NGG",cut_offset = 3):
    """
    @param seq: sequence to look through (str)
    @param guide: rna guide to search for (assumed to be in 5' --> 3' orientation)
    @param primer: "NGG" or "G" defines the primer to look for
    @param cut_offset: int the number of bases past the primer (into RNA guide) to cut at
    """
    beg = "g"
    if primer == "NGG":
        beg = "gg."
    elif primer != "G":
        raise ValueError("unsupported primer {}".format(primer))
    # turn guide into dna
    dna_guide = rna_2_dna(guide)
    
    poss_patts = [beg+dna_guide[::-1], # reverse orientation
                  dna_to_complement(beg+dna_guide[::-1]) # reverse complement
                 ]
    offset = cut_offset + (3 if primer == "NGG" else 1)
    breaks = []
    for patt in poss_patts:
        res = re.search(re.compile(patt),seq)
        if res:
            #print(patt)
            breaks.append(res.span()[0]+offset)
        # try the opp order
        res = re.search(re.compile(patt[::-1]),seq)
        if res:
            #print(patt,"rev")
            breaks.append(res.span()[1]-offset)
    if len(breaks) == 0:
        raise ValueError("Could not find guide '{}' with primer '{}'".format(guide,primer))
    elif len(breaks) == 1:
        return breaks[0]
    else:
        raise ValueError("Could not find unique guide '{}' with primer '{}'\ncuts ({})".format(guide,primer,",".join([str(b) for b in breaks])))
    
    

In [61]:
cut1 = find_guide_loc(seq,list(guides)[0])

In [62]:
cut1

156

In [63]:
print("LYPLAL1")
print(cds)
print(seq[:9]+"|"+seq[9:50])
print(seq[:23]+"|"+seq[23:50])

LYPLAL1
14..104,5270..5369,5523..5549,19205..19374,36655..36770,37615..37851
agtggcatc|agcgatggcggctgcgtcggggtcggttctgcagcgctgta
agtggcatcagcgatggcggctg|cgtcggggtcggttctgcagcgctgta


In [64]:
cut_seq = seq[:cut1]+"|"+seq[cut1:]
print(cut_seq[cut1-10:cut1+10])

acatcctccc|ctcggttac


In [65]:
cut1

156

In [66]:
for guide in guides:
    cut = find_guide_loc(seq,guide)
    cut_seq = seq[:cut]+"|"+seq[cut:]
    print("{:3d} {:20s}".format(cut,cut_seq[cut-8:cut+8]))

156 atcctccc|ctcggtt    
 83 cgcctctc|tgatctt    
 23 ggcggctg|cgtcggg    


In [110]:
for gene in set(guide_data["Gene Name"]):
    gene = gene.split()[0]
    guides = set(guide_data[guide_data["Gene Name"] == gene].Guides)
    seq = gene_data[gene_data.gene == gene].seq.iloc[0]
    cds = gene_data[gene_data.gene == gene].CDS.iloc[0]
    #print(gene)
    for guide in guides:
        cut = find_guide_loc(seq,guide)
        cut_seq = seq[:cut]+"|"+seq[cut:]
        #print("{:5d} {:20s}".format(cut,cut_seq[cut-8:cut+8]))

So now I know that the cut location seems to be working. Especially once I got the 5' --> 3' thing worked out :)

# Identify Deleted Regions

Try calculating the deleted regions for TM6SF2

## Find deleted region TM6SF2

In [70]:
gene = "TM6SF2"
guides = set(guide_data[guide_data["Gene Name"] == gene].Guides)
seq = gene_data[gene_data.gene == gene].seq.iloc[0]
cds = gene_data[gene_data.gene == gene].CDS.iloc[0]
cuts = sorted([find_guide_loc(seq,guide) for guide in guides])
cuts

[2899, 2955, 3042]

In [71]:
del_regions = [cuts[i:i+2] for i in range(len(cuts)-1)]
del_regions

[[2899, 2955], [2955, 3042]]

In [73]:
del_sizes = [r-l for l,r in del_regions]
del_sizes

[56, 87]

In [86]:
exons = [[int(c.strip("&lt;"))-1+i for i,c in enumerate(cd.split(".."))] for cd in cds.split(",")]
exons

[[86, 181],
 [2176, 2280],
 [2859, 2957],
 [3025, 3129],
 [3532, 3615],
 [4547, 4672],
 [5214, 5316],
 [5588, 5681],
 [6692, 6812],
 [8428, 8638]]

In [77]:
def calculate_exon_del(exons,del_regions):
    """
    It is assumed that none of these regions (exons or cut-sites overlap with each other)
    """
    exon_del = []
    exon_pointer = 0
    exons.sort()
    del_regions.sort()
    
    for del_region in del_regions:
        exon_del.append(0)
        dl,dr = del_region
        # find 1st exon that has right side greater than deleted region left-side
        el,er = exons[exon_pointer]
        while exon_pointer < len(exons) and er <= dl:
            exon_pointer += 1
            el,er = exons[exon_pointer]
        while er <= dr:
            exon_del[-1] += er-max(el,dl)
            exon_pointer += 1
            el,er = exons[exon_pointer]
        if el < dr:
            exon_del[-1] += dr-max(el,dl)
    return exon_del

In [87]:
calculate_exon_del(exons,del_regions)

[56, 19]

In [82]:
bit_set = ["s"]*len(seq)

In [84]:
for l,r in exons:
    for i in range(l,r+1):
        bit_set[i]+="e"
for l,r in del_regions:
    for i in range(l,r):
        bit_set[i]+="d"

In [85]:
from collections import Counter
Counter(bit_set)

Counter({'s': 7734, 'se': 1059, 'sed': 75, 'sd': 68})

Ok after accounting for off-by-one errors, I can recreate the TM6SF2 example. There are still many test cases I would like to build and test for this algorithm, because although it is just as fast as the sequence set solution for now, I am not sure how the sequence set solution will handle sequences of 10000+ bases.

## Create full results for TM6SF2

In [92]:
print(gene)
result = {"gene":gene}
guides = set(guide_data[guide_data["Gene Name"] == gene].Guides)
seq = gene_data[gene_data.gene == gene].seq.iloc[0]
cds = gene_data[gene_data.gene == gene].CDS.iloc[0]
exons = [[int(c.strip("&lt;"))-1+i for i,c in enumerate(cd.split(".."))] for cd in cds.split(",")]
result["raw_cds"] = cds.strip("&lt;")
result["full_seq"] = seq
cuts_guides = sorted([(find_guide_loc(seq,guide),guide) for guide in guides])
ord_guides = [g for _,g in cuts_guides]
ord_cuts = [c for c,_ in cuts_guides]
del_regions = [ord_cuts[i:i+2] for i in range(len(ord_cuts)-1)]
del_sizes = [r-l for l,r in del_regions]
del_exon_sizes = calculate_exon_del(exons,del_regions)
for i in range(len(cuts_guides)):
    c,g  = cuts_guides[i]
    result["guide_{}".format(i+1)] = g
    result["guide_{}_cut_index".format(i+1)] = c
for i in range(len(cuts_guides)):
    if i != 0:
        result["deletion_frag_{}".format(i)] = "guide_{},guide_{}".format(i-1,i)
        result["deletion_frag_{}_size".format(i)] = del_sizes[i-1]
        result["deletion_frag_{}_exon_size".format(i)] = del_exon_sizes[i-1]
result["total_del_size"] = sum(del_sizes)
result["total_del_exon_size"] = sum(del_exon_sizes)
result["seq_frameshift"] = result["total_del_size"]%3 != 0
result["exon_seq_frameshift"] = result["total_del_exon_size"]%3 != 0
result

TM6SF2


{'gene': 'TM6SF2',
 'raw_cds': '87..181,2177..2280,2860..2957,3026..3129,3533..3615,4548..4672,5215..5316,5589..5681,6693..6812,8429..8638',
 'full_seq': 'agcctcggcccggactcgccgccaacgaggagccgagcgccctggagaacgcgtcggggctgagccggggtccagcagccgccgctatggacatcccgccgctggccggcaagatcgcggcgctgtcgctgagcgccctcccggtgtcctacgcgctcaaccacgtctcggcgctctcgcagtgcgtgcggagaggggccagggcggccttggggacagtgggggcgggcgggggcggggaggactggaggtgggctggagggaggatgcgcggtccgcggacctggcttctgcgagccctgcgccccagctcagctccgacccggcaggattggggtgggccagaagaacacccccctctaaccggaagtccccttcccgccgggctggaggctgggatgcggccgctcctgggatgtttgtggcctgggaagaaggaatgatgagtccccacaagccagagctttggagggtttctggggttttcccagggatgcgccctgcctaggttccaaagtcaccaagggtaaaagtattgtcccagataaggccccgggtacctgactcctgctcacacacacacacacacacacacacacacacacacaccctactccattgcctgtctcaggcctgtgactttcctcttgttcagggctggctaaccatgtccctagactcagttccccctactggttccctttcctggtccctgtctcctaaccctagatcctagccctgtgggtccagccaccggatagtgggcatctgcaagcccctcgtacttctaaacccagctcaacccaagccatggggtggagagagtcttaaccatgagcctggct

## apply to all genes

In [102]:
def calculate_result(gene):
    guides = set(guide_data[guide_data["Gene Name"] == gene].Guides)
    gene = gene.split()[0]
    result = {"gene":gene}
    seq = gene_data[gene_data.gene == gene].seq.iloc[0]
    cds = gene_data[gene_data.gene == gene].CDS.iloc[0]
    exons = [[int(c.strip("&lt;"))-1+i for i,c in enumerate(cd.split(".."))] for cd in cds.split(",")]
    result["raw_cds"] = cds.strip("&lt;")
    result["full_seq"] = seq
    cuts_guides = sorted([(find_guide_loc(seq,guide),guide) for guide in guides])
    ord_guides = [g for _,g in cuts_guides]
    ord_cuts = [c for c,_ in cuts_guides]
    del_regions = [ord_cuts[i:i+2] for i in range(len(ord_cuts)-1)]
    del_sizes = [r-l for l,r in del_regions]
    del_exon_sizes = calculate_exon_del(exons,del_regions)
    for i in range(len(cuts_guides)):
        c,g  = cuts_guides[i]
        result["guide_{}".format(i+1)] = g
        result["guide_{}_cut_index".format(i+1)] = c
    for i in range(len(cuts_guides)):
        if i != 0:
            result["deletion_frag_{}".format(i)] = "guide_{},guide_{}".format(i-1,i)
            result["deletion_frag_{}_size".format(i)] = del_sizes[i-1]
            result["deletion_frag_{}_exon_size".format(i)] = del_exon_sizes[i-1]
    result["total_del_size"] = sum(del_sizes)
    result["total_del_exon_size"] = sum(del_exon_sizes)
    result["seq_frameshift"] = result["total_del_size"]%3 != 0
    result["exon_seq_frameshift"] = result["total_del_exon_size"]%3 != 0
    return result

In [105]:
pd.DataFrame([calculate_result("ADH1B")])

,gene,raw_cds,full_seq,guide_1,guide_1_cut_index,guide_2,guide_2_cut_index,deletion_frag_1,deletion_frag_1_size,deletion_frag_1_exon_size,total_del_size,total_del_exon_size,seq_frameshift,exon_seq_frameshift
0,ADH1B,"71..88,2516..2617,3218..3356,5100..5187,5285.....",atgcactcaagcagagaagaaatccacaaagactcacagtctgctg...,G*U*G*CCAAGGAAGUGGUGAAU,5351,G*U*C*UGCAGUUAACGUUGCCA,5499,"guide_0,guide_1",148,148,148,148,True,True


In [103]:
results_list = [calculate_result(gene) for gene in set(guide_data["Gene Name"])]

In [108]:
pd.DataFrame(results_list).to_excel("../outputs/47_genes.xlsx")